In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os

load_dotenv()

config = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME', 'awash_analytics'),
    'port': int(os.getenv('DB_PORT', 3306)),
    'raise_on_warnings': True
}

DATA_DIR = "../data"
CUSTOMERS_CSV = os.path.join(DATA_DIR, "awash_customers.csv")
TRANSACTIONS_CSV = os.path.join(DATA_DIR, "awash_transactions.csv")

def create_connection():
    try:
        conn = mysql.connector.connect(**config)
        print("Connected to MySQL successfully")
        return conn
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def execute_schema():
    conn = create_connection()
    if conn is None:
        return
    cursor = conn.cursor()
    with open("../sql/schema.sql", "r", encoding="utf-8") as f:
        sql_commands = f.read().split(';')
        for command in sql_commands:
            if command.strip():
                try:
                    cursor.execute(command)
                except Error as e:
                    print(f"Schema error: {e}")
    conn.commit()
    cursor.close()
    conn.close()
    print("Schema created successfully")

def load_csv_to_table(csv_path, table_name, conn):
    df = pd.read_csv(csv_path)
    print(f"Loading {len(df)} rows from {csv_path} into {table_name}...")
    
    cursor = conn.cursor()
    
    if table_name == "customers":
        columns = ["customer_id", "full_name", "phone", "address", "account_number",
                   "account_type", "balance_etb", "home_branch", "join_date"]
        df = df[columns]  # Ensure order and existence
        
        col_str = ", ".join([f"`{col}`" for col in columns])  # Proper backticks
        placeholders = ", ".join(["%s"] * len(columns))
        insert_query = f"INSERT INTO customers ({col_str}) VALUES ({placeholders})"
        
    elif table_name == "transactions":
        columns = ["transaction_id", "account_number", "date", "amount_etb",
                   "channel", "location", "merchant", "fraud_flag"]
        df = df[columns]
        
        col_str = ", ".join([f"`{col}`" for col in columns])
        placeholders = ", ".join(["%s"] * len(columns))
        insert_query = f"INSERT INTO transactions ({col_str}) VALUES ({placeholders})"
    
    # Convert rows to tuples
    data_tuples = [tuple(row) for row in df.values]
    
    try:
        cursor.executemany(insert_query, data_tuples)
        conn.commit()
        print(f"✓ {cursor.rowcount} rows successfully inserted into {table_name}")
    except Error as e:
        print(f"✗ Error loading {table_name}: {e}")
        conn.rollback()
    
    cursor.close()

def main():
    # Uncomment ONLY if you need to recreate the database from scratch
    # execute_schema()
    
    conn = create_connection()
    if conn is None:
        return
    
    load_csv_to_table(CUSTOMERS_CSV, "customers", conn)
    load_csv_to_table(TRANSACTIONS_CSV, "transactions", conn)
    
    conn.close()
    print("\nPhase 3 Complete! All data successfully loaded into MySQL.")

if __name__ == "__main__":
    main()

Connected to MySQL successfully
Loading 10000 rows from ../data\awash_customers.csv into customers...
✓ 10000 rows successfully inserted into customers
Loading 100000 rows from ../data\awash_transactions.csv into transactions...
✓ 100000 rows successfully inserted into transactions

Phase 3 Complete! All data successfully loaded into MySQL.
